In [ ]:
library(tm)
library(NLP)
library(e1071)
library(wordcloud)
DATA1

bind1=Corpus(VectorSource(DATA1$Message))
bind1=tm_map(bind1,content_transformer(tolower))
bind1=tm_map(bind1,content_transformer(removeNumbers))
bind1=tm_map(bind1,removeWords,stopwords())
bind1=tm_map(bind,content_transformer(removePunctuation))
bind1=tm_map(bind,content_transformer(stripWhitespace))


matrid=DocumentTermMatrix(bind1)

ham=subset(bind,DATA1$Category=="ham")
spam=subset(bind,DATA1$Category=="spam")

step1_train=DATA1[1:4169,]$Category
step1_test=DATA1[4169:5572,]$Category


step2_train=matrid[1:4169,]
step2_test=matrid[4169:5572,]

step3_train=bind1[1:4169]
step3_test=bind1[4169:5572]

f_data=findFreqTerms(matrid,5)

step4_train=step2_train[,f_data]
step4_test=step2_test[,f_data]
y_n =function(x)
{y=ifelse(x>0 ,1,0)
y=factor(x,levels = c(0,1),labels = c("yes","no"))
y}

step5_train=apply(step4_train,2,y_n)
step5_test=apply(step4_test,2,y_n)

p=naiveBayes(step5_train,step1_train)
c=predict(p,step5_test)
k=table(step1_test,c)


In [ ]:
library(NLP)
library(tm)

In [ ]:
library(e1071)
library(tidyverse)
library(RColorBrewer)
library(wordcloud)
library(ggplot2)  
library(beeswarm)
library(quantmod)
library(reshape2)
library(plyr)
library(scales)
library(viridis)
library(zoo)
library(dplyr)

In [ ]:
list.files(path = "../input")

In [ ]:
DATA1=read_csv('../input/sms-spam-collection-dataset/spam.csv')

In [ ]:
head(DATA1)

In [ ]:
data=DATA1[0:2]

In [ ]:
data <- sapply(data,function(row) iconv(row, "latin1", "ASCII", sub=""))
data <- data[,1:2]
data <- data.frame(data)
names(data) <- c('type','text')
data$type <- factor(data$type)
table(data$type)

In [ ]:
data

In [ ]:
bind1=Corpus(VectorSource(data$text))



In [ ]:
bind1=tm_map(bind1,content_transformer(tolower))


In [ ]:
bind1=tm_map(bind1,content_transformer(removeNumbers))


In [ ]:
bind1=tm_map(bind1,removeWords,stopwords())


In [ ]:
bind1=tm_map(bind1,content_transformer(removePunctuation))


In [ ]:
bind1=tm_map(bind1,content_transformer(stripWhitespace))

In [ ]:
bind1

In [ ]:
matrid=DocumentTermMatrix(bind1)

In [ ]:
ham=subset(bind1,data$type=="ham")
spam=subset(bind1,data$type=="spam")

In [ ]:
step1_train=data[1:4169,]$type
step1_test=data[4169:5572,]$type





In [ ]:
step2_train=matrid[1:4169,]
step2_test=matrid[4169:5572,]



In [ ]:
step3_train=bind1[1:4169]
step3_test=bind1[4169:5572]

In [ ]:
f_data=findFreqTerms(matrid,5)

In [ ]:
step4_train=step2_train[,f_data]
step4_test=step2_test[,f_data]

In [ ]:
yes_no =function(x)
{y=ifelse(x>0 ,1,0)
y=factor(x,levels = c(0,1),labels = c("yes","no"))
y}

In [ ]:
step5_train=apply(step4_train,2,yes_no)
step5_test=apply(step4_test,2,yes_no)



In [ ]:
step5_train

In [ ]:
step1_train

In [ ]:
model=naiveBayes(step5_train,step1_train)


In [ ]:
pred_val=predict(model,step5_test)


In [ ]:
con_matrix=table(step1_test,pred_val)

In [ ]:
con_matrix

In [ ]:
Accuracy =sum(diag(con_matrix))/sum(con_matrix)*100

In [ ]:
Accuracy